# 第9章: RNN, CNN

In [1]:
# import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

In [2]:
import tensorflow as tf
from tensorflow import keras

## 80. ID番号への変換
問題51で構築した学習データ中の単語にユニークなID番号を付与したい．学習データ中で最も頻出する単語に1，2番目に頻出する単語に2，……といった方法で，学習データ中で2回以上出現する単語にID番号を付与せよ．そして，与えられた単語列に対して，ID番号の列を返す関数を実装せよ．ただし，出現頻度が2回未満の単語のID番号はすべて0とせよ．

In [3]:
# 学習データ読み込み
df_train = pd.read_csv('train.txt', sep='\t')
df_train.head()

,CATEGORY,TITLE
0,b,RPT-Fitch Updates EMEA Consumer ABS Rating Cri...
1,e,"Gurlitt Wants to Return Nazi-Looted Art, Suedd..."
2,b,"UPDATE 1-Fairfax Financial, CEO probed over po..."
3,e,Angelina Jolie - Angelina Jolie Will Not Tight...
4,b,Patent Officials Cancel the Washington Redskin...


### 単語の出現回数取得

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
vectorizer = CountVectorizer()
# 計算 & 変換
train_data = vectorizer.fit_transform(df_train['TITLE'].tolist()).toarray()
feature_names = vectorizer.get_feature_names()

In [6]:
df_word_cnt = (
    pd.DataFrame(train_data, columns=feature_names)
    .sum(axis=0) # 出現回数合計
    .reset_index()
)
df_word_cnt.columns=['word', 'cnt']
df_word_cnt

,word,cnt
0,00,2
1,05,3
2,07,1
3,08,1
4,0ff,1
...,...,...
12876,zynga,3
12877,zâ,1
12878,œf,1
12879,œlousyâ,1


### 単語にID紐付け

In [7]:
df_temp = (
    df_word_cnt
    .loc[df_word_cnt['cnt'] >= 2, :] # 出現頻度が2回未満の単語は除外
    .sort_values(by='cnt', ascending=False) # 登場が多い順
    .reset_index(drop=True) # indexを順番に並べる
)
df_temp['ID'] = df_temp.index + 1 # ID列追加
df_temp

,word,cnt,ID
0,to,2870,1
1,in,1861,2
2,the,1612,3
3,of,1467,4
4,for,1358,5
...,...,...,...
7661,lip,2,7662
7662,liquidity,2,7663
7663,lira,2,7664
7664,listen,2,7665


In [8]:
df_temp[['word', 'ID']].values

array([['to', 1],
       ['in', 2],
       ['the', 3],
       ...,
       ['lira', 7664],
       ['listen', 7665],
       ['knee', 7666]], dtype=object)

In [9]:
trans_dict = {w: i for w, i in df_temp[['word', 'ID']].values}

### 関数作成

In [10]:
from typing import List

In [11]:
# 与えられた単語列に対して，ID番号の列を返す関数
def trans_text_2_ids(text: str, trans_dict: dict) -> str:
    words = text.split(' ') # 単語列をリスト化
    ret_list = []
    for w in words:
        try:
            ret_list.append(trans_dict[w])
        except KeyError: # 該当単語がなければ0
            ret_list.append(0)
    return ret_list

print(trans_text_2_ids('to in lira knee aaaaa', trans_dict))

[1, 2, 7664, 7666, 0]


## 81. RNNによる予測

### 文字列を単語列（単語のID番号のone-hot表記）に変換

In [12]:
# 学習データをID番号の列に変換
ids_train = df_train['TITLE'].apply(trans_text_2_ids, trans_dict=trans_dict).values

In [13]:
ids_train[:5]

array([list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
       list([0, 0, 1, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 0, 2495, 23, 609, 1354, 308]),
       list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 3, 0, 0, 0, 0])], dtype=object)

In [14]:
# 一番長い文言の単語数
max(map(len, ids_train))

125

In [15]:
# 長さ揃えてnumpy配列に入れる関数
def ids_2_x(ids_list):
    # 一番長い文言の単語数
    t = max(map(len, ids_list))
    temp = []
    for x in ids_list:
        # 一番長い長さに合わせる(0埋め)
        while len(x) < t:
            x.append(0)
        temp.append(x)
    return np.array(temp)

ids_2_x([[0], [1, 2], [3, 4, 5]])

array([[0, 0, 0],
       [1, 2, 0],
       [3, 4, 5]])

In [16]:
x_train = ids_2_x(ids_train)

In [17]:
x_train

array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   1, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       ...,
       [  0,   0, 461, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]])

In [18]:
x_train.shape

(10672, 125)

In [19]:
np.max(x_train)

7666

### 予測モデル作成

In [64]:
dim_w = 300
dim_h = 50
dim_v = np.max(x_train) + 1
label_num = 4

In [99]:
# h0
h_0 = tf.convert_to_tensor(np.zeros(dim_h).astype(np.float32))
h_0

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)>

In [101]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(dim_v, dim_w), # emb
    tf.keras.layers.SimpleRNN( # RNN
        units=dim_h,
        activation='tanh',
        return_sequences=False,
    ),
    tf.keras.layers.Dense(label_num, use_bias=True),
    tf.keras.layers.Softmax(),
])
model.layers[1].states[0] = h_0 # h0設定

In [103]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, None, 300)         2300100   
_________________________________________________________________
simple_rnn_29 (SimpleRNN)    (None, 50)                17550     
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 204       
_________________________________________________________________
softmax_5 (Softmax)          (None, 4)                 0         
Total params: 2,317,854
Trainable params: 2,317,854
Non-trainable params: 0
_________________________________________________________________


In [105]:
# model.variables

In [106]:
# yを計算する
y_pred = model(x_train)
y_pred

<tf.Tensor: shape=(10672, 4), dtype=float32, numpy=
array([[0.2753454 , 0.21935956, 0.31912908, 0.18616588],
       [0.27684322, 0.21956009, 0.3173631 , 0.18623355],
       [0.27076262, 0.22056854, 0.32225946, 0.18640947],
       ...,
       [0.27751908, 0.21839906, 0.31915274, 0.18492916],
       [0.2753454 , 0.21935956, 0.31912908, 0.18616588],
       [0.2753454 , 0.21935956, 0.31912908, 0.18616588]], dtype=float32)>

In [107]:
y_pred.shape

TensorShape([10672, 4])

## 82. 確率的勾配降下法による学習
確率的勾配降下法（SGD: Stochastic Gradient Descent）を用いて，問題81で構築したモデルを学習せよ．訓練データ上の損失と正解率，評価データ上の損失と正解率を表示しながらモデルを学習し，適当な基準（例えば10エポックなど）で終了させよ．

### 訓練データ、評価データ準備

In [42]:
# 学習データ読み込み
df_train = pd.read_csv('train.txt', sep='\t')
df_test = pd.read_csv('test.txt', sep='\t')
df_test.head()

,CATEGORY,TITLE
0,b,Vodafone's Service Revenue Falls as European M...
1,e,"So, Tupac Shakur's Final Words Were A Big ""F*C..."
2,t,Elephants really are intelligent: Creatures ca...
3,t,FTC says Snapchat deceived customers over 'dis...
4,b,UPDATE 3-GM says facing multiple probes into r...


In [44]:
# 特徴量
ids_train = df_train['TITLE'].apply(trans_text_2_ids, trans_dict=trans_dict).values
x_train = ids_2_x(ids_train)
ids_test = df_test['TITLE'].apply(trans_text_2_ids, trans_dict=trans_dict).values
x_test = ids_2_x(ids_test)

In [45]:
# 問70で作った関数
def trans_label_int(label: str) -> int:
    if label == 'b':
        return 0
    elif label == 't':
        return 1
    elif label == 'e':
        return 2
    elif label == 'm':
        return 3

In [46]:
# ラベル
y_train = (
    df_train
    .loc[:, 'CATEGORY']
    .apply(trans_label_int)
    .values
)
y_test = (
    df_test
    .loc[:, 'CATEGORY']
    .apply(trans_label_int)
    .values
)

### 学習

In [47]:
# クロスエントロピー損失を求める関数
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [48]:
model.compile(optimizer='SGD',
              loss=loss_fn,
              metrics=['accuracy'])

In [50]:
model.fit(
    x=x_train,
    y=y_train,
    epochs=10,
    validation_data=(x_test, y_test)
)

Epoch 1/10
334/334 [==============================] - 7s 19ms/step - loss: 1.3193 - accuracy: 0.4168 - val_loss: 1.2717 - val_accuracy: 0.4228
Epoch 2/10
334/334 [==============================] - 6s 18ms/step - loss: 1.2697 - accuracy: 0.4154 - val_loss: 1.2657 - val_accuracy: 0.4228
Epoch 3/10
334/334 [==============================] - 6s 18ms/step - loss: 1.2635 - accuracy: 0.4131 - val_loss: 1.2641 - val_accuracy: 0.4228
Epoch 4/10
334/334 [==============================] - 6s 18ms/step - loss: 1.2590 - accuracy: 0.4234 - val_loss: 1.2643 - val_accuracy: 0.3853
Epoch 5/10
334/334 [==============================] - 6s 18ms/step - loss: 1.2610 - accuracy: 0.4141 - val_loss: 1.2631 - val_accuracy: 0.4228
Epoch 6/10
334/334 [==============================] - 6s 19ms/step - loss: 1.2588 - accuracy: 0.4257 - val_loss: 1.2625 - val_accuracy: 0.4228
Epoch 7/10
334/334 [==============================] - 7s 20ms/step - loss: 1.2591 - accuracy: 0.4251 - val_loss: 1.2622 - val_accuracy: 0.4228

## 83. ミニバッチ化・GPU上での学習
問題82のコードを改変し，B事例ごとに損失・勾配を計算して学習を行えるようにせよ（Bの値は適当に選べ）．また，GPU上で学習を実行せよ．

In [51]:
batch_size = 16
model.fit(
    x=x_train,
    y=y_train,
    epochs=10,
    validation_data=(x_test, y_test),
    batch_size=batch_size,
)

Epoch 1/10
667/667 [==============================] - 8s 12ms/step - loss: 1.2597 - accuracy: 0.4142 - val_loss: 1.2620 - val_accuracy: 0.4228
Epoch 2/10
667/667 [==============================] - 7s 11ms/step - loss: 1.2595 - accuracy: 0.4130 - val_loss: 1.2625 - val_accuracy: 0.4220
Epoch 3/10
667/667 [==============================] - 7s 11ms/step - loss: 1.2592 - accuracy: 0.4161 - val_loss: 1.2627 - val_accuracy: 0.3861
Epoch 4/10
667/667 [==============================] - 7s 11ms/step - loss: 1.2591 - accuracy: 0.4129 - val_loss: 1.2615 - val_accuracy: 0.4228
Epoch 5/10
667/667 [==============================] - 7s 11ms/step - loss: 1.2592 - accuracy: 0.4149 - val_loss: 1.2625 - val_accuracy: 0.3846
Epoch 6/10
667/667 [==============================] - 8s 12ms/step - loss: 1.2591 - accuracy: 0.4172 - val_loss: 1.2615 - val_accuracy: 0.4228
Epoch 7/10
667/667 [==============================] - 7s 11ms/step - loss: 1.2592 - accuracy: 0.4098 - val_loss: 1.2614 - val_accuracy: 0.4228

In [52]:
print("利用できるGPUはあるか: "),
print(tf.config.experimental.list_physical_devices("GPU"))

利用できるGPUはあるか: 
[]


## 84. 単語ベクトルの導入
事前学習済みの単語ベクトル（例えば，Google Newsデータセット（約1,000億単語）での学習済み単語ベクトル）で単語埋め込みemb(x)を初期化し，学習せよ．